# 에이전트(Agent)

에이전트는 언어 모델과 도구를 결합하여 작업에 대해 추론하고, 사용할 도구를 결정하며, 솔루션을 향해 반복적으로 작업할 수 있는 시스템을 만듭니다.

`create_agent`는 프로덕션 수준의 에이전트 구현을 제공합니다.

## 환경 설정

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("LangChain-V1-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-V1-Tutorial


### 모델 (Model)

에이전트의 추론 엔진인 LLM 은 간단하게 `provider:model` 형식의 문자열로 지정할 수 있습니다.

In [3]:
from langchain.agents import create_agent

# 모델 식별자 문자열을 사용한 간단한 방법
agent = create_agent("openai:gpt-4.1-mini", tools=[])

하지만, 모델의 세부 설정을 위해서 다음과 같이 다양한 옵션을 사용할 수 있습니다.

In [2]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 인스턴스를 직접 초기화하여 더 세밀한 제어
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,  # 응답의 무작위성 제어
    max_tokens=1000,  # 최대 생성 토큰 수
    timeout=30,  # 요청 타임아웃(초)
)

agent = create_agent(model, tools=[])

#### 동적 모델

동적 모델은 런타임에 현재 상태와 컨텍스트를 기반으로 선택됩니다. 이를 통해 정교한 라우팅 로직과 비용 최적화가 가능합니다.

![](assets/wrap_model_call.png)

`ModelRequest`는 agent의 모델 호출 정보를 담는 dataclass로, middleware에서 요청을 검사하고 수정할 때 사용됩니다.

In [11]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

# 기본 모델과 고급 모델 정의
basic_model = ChatOpenAI(model="gpt-4.1-mini")
advanced_model = ChatOpenAI(model="gpt-4.1")


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    message_count = len(request.state["messages"])

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)


agent = create_agent(
    model=basic_model, tools=[], middleware=[dynamic_model_selection]  # 기본 모델
)

In [12]:
from langchain_teddynote.messages import stream_graph
from langchain_core.messages import HumanMessage

stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
물론입니다! 머신러닝(Machine Learning)의 동작 원리에 대해 간단히 설명해드리겠습니다.

### 머신러닝의 동작 원리

머신러닝은 데이터를 기반으로 컴퓨터가 스스로 규칙이나 패턴을 학습하여 예측이나 판단을 할 수 있도록 하는 기술입니다. 전통적인 프로그래밍이 사람이 직접 규칙을 코딩하는 것과 달리, 머신러닝은 데이터를 통해 규칙을 자동으로 학습합니다.

1. **데이터 수집**  
   머신러닝 모델이 학습할 수 있도록 다양한 양질의 데이터를 준비합니다. 예를 들어, 사진, 텍스트, 숫자 등 여러 형태가 될 수 있습니다.

2. **데이터 전처리**  
   수집한 데이터를 정제(cleaning), 정규화(normalization), 특징 추출(feature extraction) 등의 과정을 거쳐 모델 학습에 적합한 형태로 만듭니다.

3. **모델 선택**  
   문제의 성격에 맞는 머신러닝 알고리즘(예: 선형 회귀, 의사결정트리, 신경망 등)을 선택합니다.

4. **학습(Training)**  
   준비된 데이터를 모델에 입력해 모델의 내부 파라미터를 조정합니다. 모델이 데이터의 패턴을 인식하도록 하기 위해 손실 함수(loss function)를 최소화하는 방향으로 최적화를 수행합니다.

5. **평가(Evaluation)**  
   학습된 모델을 새로운 데이터(테스트 데이터)에 대해 평가하여 얼마나 잘 예측하는지 성능을 확인합니다. 일반적으로 정확도, 정밀도, 재현율, F1 점수 등 다양한 평가 지표를 사용합니다.

6. **예측 및 적용**  
   성능이 검증된 모델을 실제 환경에 적용해 새로운 데이터에 대해 예측하거나 분류 등의 작업을 수행합니다.

---

### 요약하자면:  
머신러닝은 **데이터 → 학습(패턴 인식) → 예측**의 과정을 통해 동작하며, 모델이 스스로 경험(데이터)을 통해 성능을 개선하는

수정에 활용할 **주요 속성**은 다음과 같습니다.

* `model`: 사용할 `BaseChatModel` 인스턴스
* `system_prompt`: 시스템 프롬프트 (optional)
* `messages`: 대화 메시지 리스트 (시스템 프롬프트 제외)
* `tool_choice`: tool 선택 설정
* `tools`: 사용 가능한 tool 리스트
* `response_format`: 응답 형식 지정
* `state`: 현재 agent 상태 (`AgentState`)
* `runtime`: agent runtime 정보
* `model_settings`: 추가 모델 설정 (dict)

In [25]:
@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    message_count = len(request.state["messages"][-1].content)
    print(f"글자수: {message_count}")

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        # 여러 속성 동시 변경
        new_request = request.override(
            model=advanced_model,
            system_prompt="emoji 를 사용해서 답변해줘",
            tool_choice="auto",
        )
        return handler(new_request)
    else:
        new_request = request.override(
            system_prompt="한 문장으로 간결하게 답변해줘. emoji 는 사용하지 말아줘.",
            tool_choice="auto",
            model=basic_model,
        )
        return handler(new_request)


agent = create_agent(
    model=basic_model, tools=[], middleware=[dynamic_model_selection]  # 기본 모델
)

글자수 10자 미만일 때의 응답

In [26]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="머신러닝 동작원리")]})

글자수: 9

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
머신러닝은 데이터에서 패턴을 학습해 예측이나 분류 모델을 만드는 과정입니다.

글자수 10자 이상일 때의 응답

In [ ]:
stream_graph(
    agent,
    inputs={
        "messages": [
            HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해 주세요.")
        ]
    },
)

글자수: 25

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
물론이죠! 머신러닝의 동작 원리를 간단히 설명해드릴게요 😊

1. 데이터 수집 📊  
머신러닝 모델은 먼저 학습에 사용할 데이터를 많이 모아요.

2. 데이터 전처리 🧹  
모은 데이터를 깨끗하게 정리하고, 필요한 형태로 바꿔요.

3. 모델 선정 및 학습 🧠  
적절한 알고리즘(예: 선형회귀, 결정트리 등)을 선택하고, 데이터를 이용해 모델을 학습시켜요. 즉, 데이터에서 패턴을 찾도록 해요.

4. 예측 및 평가 🔍  
학습된 모델을 이용해서 새로운 데이터에 대한 예측을 해보고, 실제 값과 비교하며 얼마나 잘 맞췄는지 평가해요.

5. 모델 개선 🔄  
평가 결과를 바탕으로 모델을 다시 학습시키거나, 데이터를 더 보강해서 성능을 높여요.

이렇게 반복하며 점점 더 똑똑한 모델을 만드는 게 머신러닝의 기본적인 동작 원리예요! 🚀

### 도구 (Tools)

도구는 에이전트에게 행동을 취할 수 있는 능력을 부여합니다. 

에이전트는 다음을 지원합니다.
- 순차적으로 여러 도구 호출
- 적절한 경우 병렬 도구 호출
- 이전 결과를 기반으로 한 동적 도구 선택
- 도구 재시도 로직 및 오류 처리
- 도구 호출 간 상태 지속성

#### 도구 정의

In [7]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"


@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    # 일부러 오류 발생
    raise Exception("날씨 정보를 가져오는 중 오류가 발생하였습니다.")


# 도구 리스트를 에이전트에 전달
agent = create_agent("openai:gpt-4.1-mini", tools=[search, get_weather])

#### 도구 오류 처리

도구 오류 처리를 커스터마이징하려면 `@wrap_tool_call` 데코레이터를 사용하여 미들웨어를 생성합니다.

In [8]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 오류를 커스텀 메시지로 처리"""
    try:
        return handler(request)
    except Exception as e:
        # 모델에 커스텀 오류 메시지 반환
        return ToolMessage(
            content=f"[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. ({str(e)})",
            tool_call_id=request.tool_call["id"],
        )


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
)

In [9]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="서울 날씨 조회해줘")]})


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. (날씨 정보를 가져오는 중 오류가 발생하였습니다.)
🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송하지만, 현재 서울의 날씨 정보를 가져오는 데 문제가 발생했습니다. 잠시 후 다시 시도해 주시겠어요?

## 프롬프트

### 시스템 프롬프트

시스템 프롬프트를 제공하여 에이전트가 작업에 접근하는 방식을 형성할 수 있습니다.

In [10]:
agent = create_agent(
    "openai:gpt-4.1-mini",
    tools=[search, get_weather],
    system_prompt="You are a helpful assistant. Be concise and accurate.",
)

#### 동적 시스템 프롬프트

런타임 컨텍스트나 에이전트 상태를 기반으로 시스템 프롬프트를 수정해야 하는 고급 사용 사례의 경우 미들웨어를 사용할 수 있습니다.

- 핵심: `request.runtime.context`

In [13]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    answer_type: str


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """사용자 역할에 따라 시스템 프롬프트 생성"""
    answer_type = request.runtime.context.get("answer_type", "default")
    print(answer_type)
    base_prompt = "You are a helpful assistant."

    if answer_type == "default":
        return f"{base_prompt} Answer in Korean. 간결하게 답변해줘."
    elif answer_type == "sns":
        return f"{base_prompt} Answer in Korean. SNS 형식으로 답변해줘."
    elif answer_type == "article":
        return f"{base_prompt} Answer in Korean. 뉴스 기사 형식으로 답변해줘."

    return base_prompt


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search],
    middleware=[user_role_prompt],
    context_schema=Context,
)

In [15]:
# 컨텍스트에 따라 시스템 프롬프트가 동적으로 설정됩니다
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(answer_type="article"),
)

article

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[뉴스 기사 형식]

머신러닝의 동작 원리, 어떻게 작동하나?

최근 인공지능 기술의 핵심으로 주목받는 머신러닝(Machine Learning)은 데이터를 통해 스스로 학습하고 예측 또는 판단을 수행하는 기술이다. 머신러닝의 동작 원리는 크게 데이터 수집, 전처리, 학습, 평가, 그리고 예측의 과정으로 나눌 수 있다.

먼저, 머신러닝은 다양한 데이터셋을 수집하는 것으로 시작한다. 이 데이터는 모델이 학습할 기초 자료가 된다. 이후 데이터의 품질을 높이기 위해 결측치 처리, 정규화 등의 전처리 작업이 이루어진다.

다음 단계는 모델 학습이다. 머신러닝 알고리즘은 주어진 데이터에서 패턴이나 규칙을 찾아 내부 수학적 모델을 구축한다. 대표적인 알고리즘으로는 회귀분석, 의사결정나무, 신경망 등이 있다. 학습 과정에서 모델은 입력 데이터와 정답 데이터(라벨)를 통해 예측 능력을 점진적으로 높인다.

학습이 완료된 모델은 평가를 거쳐 정확도, 정밀도, 재현율 등의 성능 지표를 확인한다. 평가 결과가 만족스러울 경우, 실제 데이터에 적용해 새로운 데이터를 예측하거나 분류하는데 사용된다.

이처럼 머신러닝은 데이터를 기반으로 스스로 패턴을 발견하고, 이를 통해 문제 해결 능력을 향상시키는 기술로 다양한 산업 분야에서 혁신을 이끌고 있다.

전문가들은 "머신러닝은 단순한 기술을 넘어 데이터 이해와 처리 능력의 집약체"라며, "적절한 데이터 확보와 알고리즘 선택이 성능 향상의 핵심"이라고 강조했다.

## 에이전트 호출

에이전트의 상태에 업데이트를 전달하여 에이전트를 호출할 수 있습니다. 모든 에이전트는 상태에 메시지 시퀀스를 포함합니다.

In [17]:
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(answer_type="default"),
)

default

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
머신러닝은 데이터에서 패턴을 학습해 예측이나 분류 등의 작업을 수행하는 기술입니다. 기본 원리는 다음과 같습니다.

1. 데이터 수집: 학습에 필요한 데이터를 모읍니다.
2. 데이터 전처리: 결측치 처리, 정규화 등 데이터를 학습에 적합하게 가공합니다.
3. 모델 선택: 문제 유형에 맞는 알고리즘(예: 회귀, 분류, 군집 등)을 선택합니다.
4. 학습: 모델이 주어진 데이터로부터 규칙이나 패턴을 찾아내도록 훈련합니다.
5. 평가: 학습한 모델을 새로운 데이터에 적용해 성능을 측정합니다.
6. 예측 및 응용: 학습된 모델로 실제 문제를 예측하거나 분류합니다.

즉, 머신러닝은 데이터로부터 규칙을 자동으로 발견해 문제 해결에 활용하는 과정입니다.

## 고급 개념

### 구조화된 출력

특정 형식으로 에이전트의 출력을 반환하고 싶을 때가 있습니다. LangChain은 `response_format` 매개변수를 통해 구조화된 출력 전략을 제공합니다.

#### ToolStrategy

`ToolStrategy`는 인공적인 도구 호출을 사용하여 구조화된 출력을 생성합니다. 도구 호출을 지원하는 모든 모델에서 작동합니다.

In [18]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


# 응답 스키마 정의
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(
    model="openai:gpt-4.1-mini", tools=[], response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

print(result["structured_response"])
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

name='John Doe' email='john@example.com' phone='(555) 123-4567'


#### ProviderStrategy

`ProviderStrategy`는 모델 제공자의 네이티브 구조화된 출력 생성을 사용합니다. 더 안정적이지만 네이티브 구조화된 출력을 지원하는 제공자(예: OpenAI)에서만 작동합니다.

In [ ]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="openai:gpt-4.1", response_format=ProviderStrategy(ContactInfo)
)

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

In [ ]:
result["structured_response"]

### 메모리

에이전트는 메시지 상태를 통해 대화 기록을 자동으로 유지합니다. 대화 중에 추가 정보를 기억하기 위해 커스텀 상태 스키마를 사용하도록 에이전트를 구성할 수도 있습니다.

상태에 저장된 정보는 에이전트의 단기 메모리로 생각할 수 있습니다.

#### 미들웨어를 통한 상태 정의

커스텀 상태가 특정 미들웨어 훅 및 해당 미들웨어에 연결된 도구에서 액세스해야 하는 경우 미들웨어를 사용하여 커스텀 상태를 정의합니다.

In [ ]:
from typing import Any
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware


# 커스텀 상태 스키마 정의
class CustomState(AgentState):
    user_preferences: dict


class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = []

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # 모델 호출 전 커스텀 로직
        pass


agent = create_agent("openai:gpt-4.1-mini", tools=[], middleware=[CustomMiddleware()])

# 에이전트는 이제 메시지 외에 추가 상태를 추적할 수 있습니다
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

#### `state_schema`를 통한 상태 정의

도구에서만 사용되는 커스텀 상태를 정의하는 단축 방법으로 `state_schema` 매개변수를 사용합니다.

In [ ]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict


agent = create_agent("openai:gpt-4.1-mini", tools=[], state_schema=CustomState)

result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

### 스트리밍

에이전트가 여러 단계를 실행하는 경우 시간이 걸릴 수 있습니다. 중간 진행 상황을 표시하기 위해 메시지가 발생하는 대로 스트리밍할 수 있습니다.

In [ ]:
for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "Search for AI news and summarize the findings"}
        ]
    },
    stream_mode="values",
):
    # 각 청크에는 해당 시점의 전체 상태가 포함됩니다
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif hasattr(latest_message, "tool_calls") and latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

### 미들웨어

미들웨어는 실행의 다양한 단계에서 에이전트 동작을 커스터마이징하기 위한 강력한 확장성을 제공합니다.

**미들웨어 사용 사례:**

- 모델 호출 전 상태 처리 (예: 메시지 트리밍, 컨텍스트 주입)
- 모델 응답 수정 또는 검증 (예: 가드레일, 콘텐츠 필터링)
- 커스텀 로직으로 도구 실행 오류 처리
- 상태 또는 컨텍스트에 따른 동적 모델 선택
- 커스텀 로깅, 모니터링 또는 분석 추가

## ChatOpenAI 클래스

OpenAI의 chat model API를 사용하기 위한 인터페이스입니다.

### 설치 및 설정

In [ ]:
# pip install -U langchain-openai
# 또는
# uv add langchain-openai

환경 변수 설정:

In [ ]:
# export OPENAI_API_KEY="your-api-key"

### 주요 초기화 파라미터

#### Completion 파라미터

| 파라미터 | 타입 | 설명 |
|:---|:---|:---|
| `model` | `str` | 사용할 OpenAI 모델 이름 |
| `temperature` | `float` | 샘플링 온도 |
| `max_tokens` | `int \| None` | 생성할 최대 토큰 수 |
| `logprobs` | `bool \| None` | logprobs 반환 여부 |
| `stream_options` | `dict` | 스트리밍 출력 설정 (예: `{"include_usage": True}`) |
| `use_responses_api` | `bool \| None` | Responses API 사용 여부 |

#### Client 파라미터

| 파라미터 | 타입 | 설명 |
|:---|:---|:---|
| `timeout` | `float \| Tuple[float, float] \| Any \| None` | 요청 타임아웃 |
| `max_retries` | `int \| None` | 최대 재시도 횟수 |
| `api_key` | `str \| None` | OpenAI API 키 (미지정시 `OPENAI_API_KEY` 환경변수 사용) |
| `base_url` | `str \| None` | API 요청 base URL (proxy 또는 emulator 사용시) |
| `organization` | `str \| None` | OpenAI organization ID (미지정시 `OPENAI_ORG_ID` 환경변수 사용) |

### 기본 사용법

#### 모델 초기화

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

#### 메시지 호출

In [ ]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
response = model.invoke(messages)
print(response)

응답 형식:
```python
AIMessage(
    content="J'adore la programmation.",
    response_metadata={
        "token_usage": {"completion_tokens": 5, "prompt_tokens": 31, "total_tokens": 36},
        "model_name": "gpt-4o",
        "finish_reason": "stop",
    },
    usage_metadata={"input_tokens": 31, "output_tokens": 5, "total_tokens": 36},
)
```

### 스트리밍

In [ ]:
for chunk in model.stream(messages):
    print(chunk.content, end="")

전체 메시지 수집:

In [ ]:
stream = model.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
print(full)

### 비동기 처리

In [ ]:
# 단일 호출
# await model.ainvoke(messages)

# 스트리밍
# async for chunk in model.astream(messages):
#     print(chunk.content, end="")

# 배치 처리
# await model.abatch([messages])

### Tool Calling

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


model_with_tools = model.bind_tools([GetWeather])
response = model_with_tools.invoke("What's the weather in LA?")
print(response)

병렬 tool call 비활성화:

In [ ]:
response = model_with_tools.invoke(
    "What is the weather in LA and NY?", parallel_tool_calls=False
)
print(response)

### Structured Output

In [ ]:
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int | None = Field(description="How funny the joke is, from 1 to 10")


structured_model = model.with_structured_output(Joke)
result = structured_model.invoke("Tell me a joke about cats")
print(result)

#### 메서드 옵션

* `json_schema`: OpenAI Structured Output API 사용 (기본값)
* `function_calling`: OpenAI tool-calling API 사용
* `json_mode`: OpenAI JSON mode 사용

In [ ]:
structured_model = model.with_structured_output(
    schema=Joke, method="json_schema", include_raw=True, strict=True
)

### JSON Mode

In [ ]:
json_model = model.bind(response_format={"type": "json_object"})
response = json_model.invoke(
    "Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]"
)
print(response.content)

### 이미지 입력

In [ ]:
import base64
import httpx
from langchain_core.messages import HumanMessage

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)

response = model.invoke([message])
print(response.content)

### Token 사용량 추적

In [ ]:
response = model.invoke(messages)
print(response.usage_metadata)
# {"input_tokens": 28, "output_tokens": 5, "total_tokens": 33}

스트리밍시:

In [ ]:
stream = model.stream(messages, stream_usage=True)
full = next(stream)
for chunk in stream:
    full += chunk
print(full.usage_metadata)

### Logprobs

In [ ]:
logprobs_model = model.bind(logprobs=True)
response = logprobs_model.invoke(messages)
print(response.response_metadata["logprobs"])

### OpenAI 호환 API 사용

In [ ]:
# LM Studio 예시
# model = ChatOpenAI(
#     base_url="http://localhost:1234/v1",
#     api_key="lm-studio",
#     model="mlx-community/QwQ-32B-4bit",
#     extra_body={"ttl": 300}
# )

# vLLM 예시
# model = ChatOpenAI(
#     base_url="http://localhost:8000/v1",
#     api_key="EMPTY",
#     model="meta-llama/Llama-2-7b-chat-hf",
#     extra_body={"use_beam_search": True, "best_of": 4}
# )

### 파라미터 구분

**`model_kwargs` 사용:**
* 표준 OpenAI API 파라미터
* 최상위 요청 payload에 병합되는 파라미터

**`extra_body` 사용:**
* OpenAI 호환 provider의 커스텀 파라미터
* `extra_body` 키 하위에 중첩되는 파라미터

In [ ]:
# model_kwargs 예시
# model = ChatOpenAI(
#     model="gpt-4o",
#     model_kwargs={
#         "stream_options": {"include_usage": True},
#         "max_completion_tokens": 300,
#     }
# )

# extra_body 예시
# model = ChatOpenAI(
#     base_url="http://localhost:8000/v1",
#     extra_body={
#         "use_beam_search": True,
#         "best_of": 4,
#     }
# )

### Prompt Caching

반복적인 prompt가 있는 대용량 애플리케이션에서 cache hit rate 향상:

In [ ]:
response = model.invoke(messages, prompt_cache_key="example-key-a")

# 동적 cache key
dynamic_suffix = "user123"
cache_key = f"example-key-{dynamic_suffix}"
response = model.invoke(messages, prompt_cache_key=cache_key)